In [1]:
! pip install git+https://github.com/openai/whisper.git -q

In [2]:
! pip install gradio -q

In [1]:
import gradio as gr
import os
import sys
import subprocess


In [2]:
import whisper

model = whisper.load_model("small")

In [3]:
def video2mp3(video_file, output_ext="mp3"):
    filename, ext = os.path.splitext(video_file)
    subprocess.call(["ffmpeg", "-y", "-i", video_file, f"{filename}.{output_ext}"],
                    stdout=subprocess.DEVNULL,
                    stderr=subprocess.STDOUT)
    return f"{filename}.{output_ext}"


In [4]:
input_video = '12 movie youtubeshorts.mp4'

In [6]:
audio_file = video2mp3(input_video)

In [7]:
from IPython.display import Audio
Audio(audio_file)

In [8]:
def translate(audio):

    options = dict(beam_size=5, best_of=5)
    translate_options = dict(task="translate", **options)
    result = model.transcribe(audio_file,**translate_options)
    return result

In [9]:
result = translate(audio_file)

c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [10]:
print(result)

{'text': " I don't even have time for love. I just have to focus on my studies. I have a lot of responsibilities.", 'segments': [{'id': 0, 'seek': 0, 'start': 0.0, 'end': 3.0, 'text': " I don't even have time for love.", 'tokens': [50364, 286, 500, 380, 754, 362, 565, 337, 959, 13, 50514], 'temperature': 0.0, 'avg_logprob': -0.5750842094421387, 'compression_ratio': 1.2, 'no_speech_prob': 0.27523961663246155}, {'id': 1, 'seek': 0, 'start': 4.0, 'end': 6.0, 'text': ' I just have to focus on my studies.', 'tokens': [50564, 286, 445, 362, 281, 1879, 322, 452, 5313, 13, 50664], 'temperature': 0.0, 'avg_logprob': -0.5750842094421387, 'compression_ratio': 1.2, 'no_speech_prob': 0.27523961663246155}, {'id': 2, 'seek': 0, 'start': 8.0, 'end': 10.0, 'text': ' I have a lot of responsibilities.', 'tokens': [50764, 286, 362, 257, 688, 295, 16190, 13, 50864], 'temperature': 0.0, 'avg_logprob': -0.5750842094421387, 'compression_ratio': 1.2, 'no_speech_prob': 0.27523961663246155}], 'language': 'hi'}


In [11]:
print(result["text"])

 I don't even have time for love. I just have to focus on my studies. I have a lot of responsibilities.


In [12]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open(input_video,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [13]:
import whisper.utils
print(dir(whisper.utils))

['Callable', 'List', 'Optional', 'ResultWriter', 'SubtitlesWriter', 'TextIO', 'WriteJSON', 'WriteSRT', 'WriteTSV', 'WriteTXT', 'WriteVTT', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'compression_ratio', 'exact_div', 'format_timestamp', 'get_end', 'get_start', 'get_writer', 'json', 'make_safe', 'optional_float', 'optional_int', 'os', 're', 'str2bool', 'sys', 'system_encoding', 'zlib']


In [14]:
output_dir = "/content/"
audio_path = os.path.splitext(audio_file)[0]
vtt_file = os.path.join(output_dir, audio_path + ".vtt")

# Manually write the VTT file
with open(vtt_file, "w", encoding="utf-8") as vtt:
    vtt.write("WEBVTT\n\n")
    for segment in result["segments"]:
        start = segment["start"]
        end = segment["end"]
        text = segment["text"]

        # Convert timestamps to VTT format (HH:MM:SS.mmm)
        start_time = f"{int(start // 3600):02}:{int(start % 3600 // 60):02}:{int(start % 60):02}.{int((start % 1) * 1000):03}"
        end_time = f"{int(end // 3600):02}:{int(end % 3600 // 60):02}:{int(end % 60):02}.{int((end % 1) * 1000):03}"

        vtt.write(f"{start_time} --> {end_time}\n{text}\n\n")


In [17]:
# Ensure we use the VTT file produced earlier and build an output filename
subtitle = vtt_file  # vtt_file is created in a previous cell
output_video = os.path.splitext(input_video)[0] + "_subtitled.mp4"

# Run ffmpeg to burn subtitles into the video (quote paths to handle spaces)
exit_code = os.system(f'ffmpeg -y -i "{input_video}" -vf subtitles="{subtitle}" "{output_video}"')
print(f"FFmpeg exit code: {exit_code}")


FFmpeg exit code: 0


In [18]:
import os

if not os.path.exists(input_video.strip('"')):  # Remove quotes if added earlier
    raise FileNotFoundError(f"Input video file '{input_video}' not found.")


In [ ]:
os.system(f'ffmpeg -i "{input_video}" -vf subtitles="{subtitle}" "{output_video}"')


In [ ]:
if not os.path.exists(subtitle.strip('"')):
    raise FileNotFoundError(f"Subtitle file '{subtitle}' not found.")


In [142]:
ffmpeg -i "12 movie youtubeshorts.mp4" -vf subtitles="12 movie youtubeshorts.vtt" "12 movie youtubeshorts_subtitled.mp4"


SyntaxError: invalid syntax (4249174365.py, line 1)

In [143]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open(output_video,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

FileNotFoundError: [Errno 2] No such file or directory: '12 movie youtubeshorts_subtitled.mp4'